# Storm Petrels detection with **warbleR**
[warbleR](https://github.com/cran/warbleR/) is a very handy R package for bioacoustics. Full description can be found in [this](https://besjournals.onlinelibrary.wiley.com/doi/epdf/10.1111/2041-210X.12624) paper. Among other features, the package offers `autodetec` routine that can detect, after tuning of parameters, relevant acoustic signals. After extraction of *start* and *end* of a potential bird call, we pass this information to `specan` function that calculates audio features described in [documentation](https://www.rdocumentation.org/packages/warbleR/versions/1.1.12/topics/specan). 

## The goal
We are interested in what how many hypothetical calls are detected by **warbleR**. The R code can be found in 

## Load Python modules
`read_labels` offers a routine to load Excel spreadsheet and takes care of time conversion.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
from scipy.io import wavfile
from birdutils import read_labels

## Data paths

In [2]:
warbler_audio_features_path = '/mnt/data/Birdman/results/warbler/features.csv'
bird_calls_labels_path = '/mnt/data/Birdman/sthelena_labels.xls'

## Load the data
Load the data. I rename `start` and `end` column so that they match naming convention in the Excel label file. 

In [3]:
labels_dict = read_labels(bird_calls_labels_path)
labels = pd.concat(labels_dict, axis=0, ignore_index=True)
features = pd.read_csv(warbler_audio_features_path)
features = features.rename(columns={'start': 'Time Start', 'end': 'Time End'})

In [4]:
print('Number of signal found with warbleR:', len(features))

Number of signal found with warbleR: 1084


In [5]:
print('Number of labels per species:')
labels['Species'].value_counts()

Number of labels per species:


Storm Petrel    367
Brown Noddy      21
Name: Species, dtype: int64

## warbleR audio features
These audio features are calculated per region of interest, defined as the period between start and end of the call. The minimal length of the call has been defined as $\frac{1}{10}$ second while max is $1$ second.

In [6]:
features.head(3).T

,0,1,2
sound.files,STHELENA-02_20140605_200000_10,STHELENA-02_20140605_200000_10,STHELENA-02_20140605_200000_10
selec,1,2,3
duration,0.0804259,0.137176,0.317176
meanfreq,2.88295,2.9662,2.8024
sd,1.21045,1.1945,1.05719
freq.median,2.85477,3.05109,2.85174
freq.Q25,2.28216,1.81752,2.16404
freq.Q75,3.05394,3.62774,3.09779
freq.IQR,0.771784,1.81022,0.933754
time.median,0.0301641,0.0823125,0.16288


`select` refers to the detected region on a [spectrogram](https://en.wikipedia.org/wiki/Spectrogram), in this case:

![Spectrogram](resources/STHELENA-02_20140605_200000_10.jpeg)

The red dashed lines denote start and end of the detected call. Mind that algorithm will miss some calls (false negative, analogous to Type I error), while also detect calls where there none (false positive, analogous to Type 2 error) - more on these on [Wikipedia](https://en.wikipedia.org/wiki/False_positives_and_false_negatives). Two caveats:
* The `autodetect` routine removes background and potentially also some bird calls. Do not try to infer true number of bird calls from such preprocessed spectrogram.
* The labels are not always accurate. It's easy to miss calls, especially if they're faint.

![false_positive_negative](resources/fp_fn_errors.png)

From *The Essential Guide to Effect Sizes: Statistical Power, Meta-Analysis, and the Interpretation of Research Results*

## Labels from the Excel file

In [7]:
labels_dict['STHELENA-02_20140605_200000_10'].head(5)

,Date,File Name,Type of Call,Time Start,Time End,Species,Notes
0,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,16.142,17.433,Storm Petrel,NaN
1,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,20.016,21.630,Storm Petrel,NaN
2,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,21.953,23.244,Storm Petrel,NaN
3,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,39.386,40.354,Storm Petrel,NaN
4,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,46.165,47.779,Storm Petrel,NaN


## The raw audio
In the example we have focused on first 50 seconds of this recording: STHELENA-02_20140605_200000_10.wav The widget below lets you play the audio

In [8]:
import IPython
IPython.display.Audio("resources/STHELENA-02_20140605_200000_10.mp3")

## Observations
* Around 9-12s: calls not present in labels. `warbleR` detects a call around 11s
* 16-17s: present in labels, not in `warbleR` 
* 20-23s: present in both
* 39-40s: present in both
* 41-42s: not present in both
* 46-47s: present in both

## Find overlap
Let's find how much overlap is between what `warbleR` detects and labels - and vice versa. The function finds if there is an overlap between the two. 

Overlap between labels and `warbleR` will tell us how many calls were detected (overlap = True) and how many missed (overlap = False). In turn, we will get **false negative** rate of a sort. 

Overlap between `warbleR` and labels will tell us how many calls were detected (overlap = True) and how many identified while not present in labels (overlap = False). In turn, we will get **false positive** rate of a sort. 

In [9]:
features['overlap'] = False
features['storm_petrel'] = False
labels['overlap'] = False
labels['storm_petrel'] = False

for filename in labels_dict.keys():
    df_label = labels[labels['File Name'] == filename]
    df_feat = features[features['sound.files'] == filename]
    for index1, row_label in df_label.iterrows():
        for index2, row_feat in df_feat.iterrows():
            feat_start = row_feat['Time Start']
            feat_end = row_feat['Time End']
            label_start = row_label['Time Start']
            label_end = row_label['Time End']
            overlap = (label_start <= feat_end) and (label_end >= feat_start)
            if overlap:
                labels.at[index1, 'overlap'] = True
                features.at[index2, 'overlap'] = True
                if row_label['Species'] == 'Storm Petrel':
                    labels.at[index1, 'storm_petrel'] = True
                    features.at[index2, 'storm_petrel'] = True

An alternative way to find overlap, on grouped objects - not used. It's much faster and concise, as it uses apply and vectorised operations, but might be difficult to read.

In [10]:
def find_overlap(group, **kwargs):
    """
    The double asterisk form of **kwargs is used to pass a keyworded, variable-length argument dictionary to a function. 
    """
    name = os.path.splitext(group.name)[0]
    
    label = kwargs['label']
    
    labels = label[name]
    label_start = labels['Time Start']
    label_end = labels['Time End']

    result = []

    for index, row in group.iterrows():
        start = row['Time Start']
        end = row['Time End']
        overlap = any((label_start <= end) & (label_end >= start))
        result.append(overlap)
    group['overlap'] = result

    return group

## Results for STHELENA-02_20140605_200000_10.
Let's study the first 50 seconds from the previous example. We are first selecting specific recording and then putting an upper limit on End Time.

In [11]:
warbler_labels_overlap_10 = features[features['sound.files'] == 'STHELENA-02_20140605_200000_10']
labels_warbler_overlap_10 = labels[labels['File Name'] == 'STHELENA-02_20140605_200000_10']
warbler_labels_overlap_10_50s = warbler_labels_overlap_10[warbler_labels_overlap_10['Time End'] < 50]
labels_warbler_overlap_10_50s = labels_warbler_overlap_10[labels_warbler_overlap_10['Time End'] < 50]

To avoid clutter, we select only a subset of columns

In [12]:
warbler_columns_no_features = ['sound.files', 'selec', 'duration', 'Time Start', 'Time End', 'overlap', 'storm_petrel']
warbler_labels_overlap_10_50s[warbler_columns_no_features]

,sound.files,selec,duration,Time Start,Time End,overlap,storm_petrel
0,STHELENA-02_20140605_200000_10,1,0.080426,11.183444,11.263869,False,False
1,STHELENA-02_20140605_200000_10,2,0.137176,20.218882,20.356058,True,True
2,STHELENA-02_20140605_200000_10,3,0.317176,20.560320,20.877496,True,True
3,STHELENA-02_20140605_200000_10,4,0.139738,20.964070,21.103808,True,True
4,STHELENA-02_20140605_200000_10,5,0.173988,21.162570,21.336558,True,True
5,STHELENA-02_20140605_200000_10,6,0.172363,21.399007,21.571371,True,True
6,STHELENA-02_20140605_200000_10,7,0.152176,22.161007,22.313183,True,True
7,STHELENA-02_20140605_200000_10,8,0.261738,22.463632,22.725371,True,True
8,STHELENA-02_20140605_200000_10,9,0.134551,22.796882,22.931433,True,True
9,STHELENA-02_20140605_200000_10,10,0.142676,22.998757,23.141433,True,True


Calculate `True Positive (tp)`  and `False Positive (fp)` rates.

In [13]:
Tp = sum(warbler_labels_overlap_10_50s['storm_petrel'])
Fp = len(warbler_labels_overlap_10_50s) - Tp
print('True positive in first 50s of STHELENA-02_20140605_200000_10:', Tp)
print('False positive in first 50s of STHELENA-02_20140605_200000_10:', Fp)

True positive in first 50s of STHELENA-02_20140605_200000_10: 17
False positive in first 50s of STHELENA-02_20140605_200000_10: 2


In [14]:
labels_warbler_overlap_10_50s

,Date,File Name,Type of Call,Time Start,Time End,Species,Notes,overlap,storm_petrel
52,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,16.142,17.433,Storm Petrel,NaN,False,False
53,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,20.016,21.630,Storm Petrel,NaN,True,True
54,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,21.953,23.244,Storm Petrel,NaN,True,True
55,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,39.386,40.354,Storm Petrel,NaN,True,True
56,2014-06-05,STHELENA-02_20140605_200000_10,Flight Call,46.165,47.779,Storm Petrel,NaN,True,True


In [15]:
fn = len(labels_warbler_overlap_10_50s) - sum(labels_warbler_overlap_10_50s['storm_petrel'])
print('False negative in first 50s of STHELENA-02_20140605_200000_10:', fn)

False negative in first 50s of STHELENA-02_20140605_200000_10: 1


## Results
Now that we have seen the algorithm at works on a small example, we can calculate the metrics for the complete labelled set. Knowing `tp`, `fp` and `fn`, we can calculate elementary metrics: 
* Accuracy and Precision: $\frac{T_p}{T_p+F_p}$ In this case they are one and the same, as we don't have `True Negative` scores.
* Recall: $\frac{T_p}{T_p+F_n}$

### Overall

In [16]:
Tp = sum(features['storm_petrel'])
Fp = len(features) - Tp
Fn = len(labels) - sum(labels['storm_petrel'])

print('True positive:', Tp)
print('False positive:', Fp)
print('False negative:', Fn)
print('Accuracy / Precision: {:.2f}%'.format(Tp / (Tp + Fp) * 100))
print('Recall: {:.2f}%'.format(Tp / (Tp + Fn) * 100))

True positive: 852
False positive: 232
False negative: 137
Accuracy / Precision: 78.60%
Recall: 86.15%


### Brown Noddy
There are very few calls from brown noddy

In [17]:
brown_noddy = labels[labels['Species'] == 'Brown Noddy']
brown_noddy

,Date,File Name,Type of Call,Time Start,Time End,Species,Notes,overlap,storm_petrel
32,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,312.824,319.927,Brown Noddy,NaN,False,False
33,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,322.187,326.383,Brown Noddy,NaN,False,False
34,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,329.612,336.391,Brown Noddy,Call fades out,False,False
36,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,386.107,394.501,Brown Noddy,NaN,False,False
38,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,412.257,414.194,Brown Noddy,NaN,False,False
44,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,492.319,495.548,Brown Noddy,NaN,False,False
51,2014-06-05,STHELENA-02_20140605_200000_1,Roosting,545.909,554.626,Brown Noddy,NaN,False,False
135,2014-06-05,STHELENA-02_20140605_200000_2,Roosting,0.000,3.551,Brown Noddy,NaN,True,False
137,2014-06-05,STHELENA-02_20140605_200000_2,Roosting,12.913,19.047,Brown Noddy,NaN,True,False
150,2014-06-05,STHELENA-02_20140605_200000_2,Roosting,65.696,68.118,Brown Noddy,NaN,False,False


In [18]:
brown_noddy_overlap = sum(brown_noddy['overlap'])
print('{} out of {} of Brown Noddy calls have been identified by warbleR, which is {:.2f}% of total'.
      format(brown_noddy_overlap, len(brown_noddy), brown_noddy_overlap / len(brown_noddy) * 100))

8 out of 21 of Brown Noddy calls have been identified by warbleR, which is 38.10% of total


Since there are only 21 calls, we cannot draw any meaningful conclusions. It's clear though that the algorithm, after some tuning, is rather storm petrels specific: *storm petrels* score 78% while *brown noddy* only 38%.

# False positives? 

Are the numbers any good? It's a matter of what we are looking to achieve. This basic algorithm has higher recall than precision. 78% precision tells us that, statistically speaking, it finds more calls then there actually are - in this case it found 232 more calls than in labels. Some on them will be simply mistakes, while some will be actual calls that were missed in labels. It's worth looking into these. Please note that a single label in Excel can be easily translated into, say, 6 calls picked up by the algorithm, if it consitutes of 6 consecutive calls.

Also, the algorithm has to not bad recall (sensitivity): once it detects the signal, there's 86% probability that it's a storm petrel (in these conditions)

## False positive examples

In [19]:
fp_df = features[features['storm_petrel'] == False][warbler_columns_no_features]
fp_df.head(5)

,sound.files,selec,duration,Time Start,Time End,overlap,storm_petrel
0,STHELENA-02_20140605_200000_10,1,0.080426,11.183444,11.263869,False,False
15,STHELENA-02_20140605_200000_10,16,0.092676,40.465197,40.557873,False,False
19,STHELENA-02_20140605_200000_10,20,0.097801,137.392645,137.490446,False,False
20,STHELENA-02_20140605_200000_10,21,0.152426,161.193335,161.345761,False,False
21,STHELENA-02_20140605_200000_10,22,0.122613,192.071213,192.193827,False,False


### Path to audio data
The stereo recordings have been converted to mono and downsampled to 16khz for ease of experimenting. In future we might use original recordings, but my experience tells other factors play much more important role. We leave fine-tuning for later.

In [20]:
path_to_storm_petrels = '/mnt/data/Birdman/16khz/'
output_dir = os.path.join(path_to_storm_petrels, 'fp')

`extract_call` takes call metadata, extracts the relevant audio piece and saves it to audio file.

In [ ]:
def extract_call(call, **kwargs):
    filename = call['sound.files'] + '.wav'
    path = os.path.join(kwargs['path'], filename )
    rate, data = wavfile.read(path)
    
    start = int((call['Time Start'] - kwargs['buffer']) * rate)
    end = int((call['Time End'] + kwargs['buffer']) * rate)
    data = data[start:end]
    
    if 'out' in kwargs:
        os.makedirs(kwargs['out'], exist_ok=True)
        name = call['sound.files'] + '_' + str(call['selec']) + '.wav'
        outpath = os.path.join(kwargs['out'], name)
        wavfile.write(outpath, rate, data)
    
    return data

In [ ]:
result = fp_df.apply(extract_call, axis=1, path=path_to_storm_petrels, buffer=0.25, out=)

In [ ]:
features.to_csv('features_warbler.csv')

## Next steps
Now that we have extracted the supposedly false positives, it would be great to see for what they really are - truly false positives of perhaps missed calls? To answer that we have at least two options:
* Listen to the recordings
* Check spectrograms

I am going to do both and report the results later.

**Cheers!**

Lukasz